In [ ]:
# !pip install minerl

In [1]:
# from google.colab import drive
import minerl
import gym
import cv2
import numpy as np

import collections
import matplotlib.pyplot as plt
import pickle
import json

import numpy as np

import pickle

import os
os.environ['MINERL_DATA_ROOT']="C:\\Users\\danie\\Documents\\Uni\\Masters-Dissertation\\data"

c:\users\danie\appdata\local\programs\python\python36\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [2]:
import sys
sys.path.append("./functions")

from DataHelper import parse_demo, parse_sequences, get_unique_actions,get_unique_place_actions
from Utils import save_data, load_data
from ActionCombos import get_unique_angles, get_all_action_combos

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
environment = "Treechop"
resources_folder = 'new-resources'

In [6]:
!python -m minerl.data.download "MineRLTreechop-v0"

C:\Users\danie\AppData\Local\Programs\Python\Python37\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
C:\Users\danie\AppData\Local\Programs\Python\Python37\lib\runpy.py:125: RuntimeWarning: 'minerl.data.download' found in sys.modules after import of package 'minerl.data', but prior to execution of 'minerl.data.download'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2020-11-28 19:51:00 LAPTOP-J1KMG6CE __main__[7684] INFO Downloading experiment MineRLTreechop-v0 to C:\Users\danie\Documents\Uni\Masters-Dissertation\data
2020-11-28 19:51:00 LAPTOP-J1KMG6CE __main__[7684] WARNING C:\Users\danie\Documents\Uni\Masters-Dissertation\data\MineRLTreechop-v0 exists - skipping re-download!


In [4]:
data = parse_demo('MineRL{0}-v0'.format(environment))

100%|██████████████████████████████████████████████████████████████████████████████| 209/209 [00:00<00:00, 9512.96it/s]


In [5]:
parsed_sequences = parse_sequences(data)

In [6]:
unique_actions = get_unique_actions(data[0]['action'])

In [7]:
unique_angles = get_unique_angles(parsed_sequences)

In [ ]:
#only valid for navigate
unique_placements = get_unique_place_actions(parsed_sequences)
save_data("../{0}/unique_placements_{0}.sav".format(resources_folder,environment.lower()),unique_placements)

In [8]:
combos = get_all_action_combos(unique_actions,unique_angles)

In [ ]:
save_data("../{0}/parsed_data.sav".format(resources_folder), parsed_sequences)
save_data("../{0}/unique_actions_{0}.sav".format(resources_folder,environment.lower()),unique_actions_dataset)
save_data("../{0}/unique_angles_{0}.sav".format(resources_folder,environment.lower()),unique_angles)

In [ ]:
save_data("../{0}/action_combos_{1}.sav".format(resources_folder, environment.lower()), parsed_sequences)